# Imports used in the project.

In [105]:
from dataclasses import dataclass, astuple
import os
import pathlib
import base64
from typing import *
import tensorflow as tf
import keras as k

import cv2
import numpy as np
import io
import keras
from PIL import Image
import xmltodict
import pylab as pl

# Data Sets.
- first set : [Playing Cards Labelized Dataset](https://www.kaggle.com/hugopaigneau/playing-cards-dataset).
- second set : [Playing Cards](https://www.kaggle.com/vdntdesai11/playing-cards).

#### Parsing Classes

In [106]:
Paths: Dict[str, str] = {
  'resources': 'resources/',
  'yolo': 'resources/cards/yolo-labeled/',
  'xml': 'resources/cards/xml-labeled/',
  'record': 'resources/cards/combined_record'
}

class FileReader(object):
  @classmethod
  def read(cls, filepath: str) -> bytes:
    with tf.io.gfile.GFile(filepath, 'rb') as file: return file.read()
class ImageReader(FileReader):
  @classmethod
  def encoded_file(cls, imagepath: str) -> bytes:
    raw = super().read(imagepath)
    return base64.b85encode(raw).decode('utf-8')

  @classmethod
  def encoded(cls, image: Image.Image) -> bytes:
    with io.BytesIO() as buffer:
      image.save(buffer, image.format)
      return base64.b85encode(buffer.getvalue())

  @classmethod
  def decoded(cls, imageraw: str) -> Image.Image:
    return Image.open(io.BytesIO(base64.b85decode(imageraw)))

  @classmethod
  def read(cls, imagepath: str) -> Image.Image:
    return Image.open(io.BytesIO(super().read(imagepath)))

  @classmethod
  def read_resized(cls, imagepath: str, size: Tuple[int, int] = (400, 400)) -> Image.Image:
    image = cls.read(imagepath)
    new = image
    new = new.resize(size, Image.ANTIALIAS)
    new.format = image.format
    return new

class FeatureProvider(object):
  @classmethod
  def i64(cls, value) -> tf.train.Feature:
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
  @classmethod
  def i64_list(cls, value) -> tf.train.Feature:
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
  @classmethod
  def u8(cls, value) -> tf.train.Feature:
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
  @classmethod
  def u8_list(cls, value) -> tf.train.Feature:
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
  @classmethod
  def f32(cls, value) -> tf.train.Feature:
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
  @classmethod
  def f32_list(cls, value) -> tf.train.Feature:
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
class RecordProvider(FeatureProvider):
  @classmethod
  def record(cls, features: Dict[str, tf.train.Feature]) -> tf.train.Example:
    return tf.train.Example(features=tf.train.Features(feature=features))

class FeatureWriter(object):
  @staticmethod
  def write(records: Iterable[tf.train.Example], target: str):
    with tf.io.TFRecordWriter(f"{target}.record") as writer:
      for record in records: writer.write(record.SerializeToString())

  @staticmethod
  def combine_write(record_iterators: Iterable[Iterable[tf.train.Example]], target: str, total: int = None):
    with tf.io.TFRecordWriter(f"{target}.record") as writer:
      for (index, record) in enumerate((record for iterator in record_iterators for record in iterator), start=1):
        if (index % 100) == 0: print(f"{index}{f'/{total}' if total else ''} records written.")
        writer.write(record.SerializeToString())

class FeatureReader(object):
  @classmethod
  def read(cls, source: str) -> tf.data.TFRecordDataset:
    return tf.data.TFRecordDataset(f"{source}.record")

## Tensorflow Record preparation

### Parsing records from 'Playing Cards Labelized Dataset' dataset

#### Parse Classes

In [107]:
Classes: Dict[str, str] = {
  'As': 1, 'Ac': 13, 'Ad': 26, 'Ah': 39,
  'Ks': 2, 'Kc': 14, 'Kd': 27, 'Kh': 40,
  'Qs': 3, 'Qc': 15, 'Qd': 28, 'Qh': 41,
  'Js': 4, 'Jc': 16, 'Jd': 29, 'Jh': 42,
  '10s': 5, '10c': 17, '10d': 30, '10h': 43,
  '9s': 6, '9c': 18, '9d': 31, '9h': 44,
  '8s': 7, '8c': 19, '8d': 32, '8h': 45,
  '7s': 8, '7c': 20, '7d': 33, '7h': 46,
  '6s': 9, '6c': 21, '6d': 34, '6h': 47,
  '5s': 10, '5c': 22, '5d': 35, '5h': 48,
  '4s': 11, '4c': 23, '4d': 36, '4h': 49,
  '3s': 12, '3c': 24, '3d': 37, '3h': 50,
  '2s': 25, '2c': 38, '2d': 51, '2h': 52,
}

Xml = Dict[str, Union['Xml', str]]
class XmlReader(FileReader):
  @classmethod
  def read(cls, xmlpath: str) -> Xml:
    return xmltodict.parse(super().read(xmlpath))
class XMlRecordProvider(RecordProvider):
  @classmethod
  def to_record(cls, image: Image.Image, xml: Xml) -> tf.train.Example:
    return cls.record(cls._parse(image, xml))

  @classmethod
  def _parse(cls, image: Image.Image, xml: Xml):
    filename = xml['annotation']['filename']
    (width, height) = map(int, (xml['annotation']['size']['width'], xml['annotation']['size']['height']))
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    texts = []
    labels = []
    for card in xml['annotation']['object']:
      xmins.append(int(card['bndbox']['xmin']) / width)
      xmaxs.append(int(card['bndbox']['xmax']) / width)
      ymins.append(int(card['bndbox']['ymin']) / height)
      ymaxs.append(int(card['bndbox']['ymax']) / height)
      texts.append(card['name'].encode('utf8'))
      labels.append(Classes[card['name']])

    return {
      'name': cls.u8(filename.encode('utf8')),
      'encoded': cls.u8(ImageReader.encoded(image)),
      'format': cls.u8(image.format.encode('utf8')),
      'height': cls.i64(image.height),
      'width': cls.i64(image.width),
      'cards/bbox/xmin': cls.f32_list(xmins),
      'cards/bbox/xmax': cls.f32_list(xmaxs),
      'cards/bbox/ymin': cls.f32_list(ymins),
      'cards/bbox/ymax': cls.f32_list(ymaxs),
      'cards/class/text': cls.u8_list(texts),
      'cards/class/label': cls.i64_list(labels),
    }

#### Parsing process

In [108]:
def xml_pathfinder(extension: str) -> pathlib.Path:
  return pathlib.Path(Paths['xml']).glob(f"*.{extension}")
def xml_record(data: Tuple[Image.Image, Xml]) -> tf.train.Example:
  return XMlRecordProvider.to_record(*data)
def find_xml_record_count():
  return len(list(xml_pathfinder('jpg')))

images = map(ImageReader.read_resized, xml_pathfinder('jpg'))
xmls = map(XmlReader.read, xml_pathfinder('xml'))

xml_count = find_xml_record_count()
xml_records = map(xml_record, zip(images, xmls))

### Parsing records from 'Playing Cards' dataset

#### Parse Classes

In [109]:
@dataclass
class Yolo(object):
  classname: str
  x_center: float
  y_center: float
  width: float
  height: float

  @classmethod
  def from_bbox(cls, bbox: 'BBox') -> 'Yolo':
    (classname, xmin, xmax, ymin, ymax) = astuple(bbox)
    w = xmax - xmin
    h = ymax - ymin
    x = (xmax + xmin) / 2.0
    y = (ymax + ymin) / 2.0
    return cls(classname, x, y, w, h)
@dataclass
class BBox(object):
  classname: str
  xmin: float
  xmax: float
  ymin: float
  ymax: float

  @classmethod
  def from_yolo(cls, yolo: Yolo) -> 'BBox':
    (classname, x_center, y_center, width, height) = astuple(yolo)
    xmax = width / 2 + x_center
    xmin = 2 * x_center - xmax

    ymax = height / 2 + y_center
    ymin = 2 * y_center - ymax
    return cls(classname, xmin, xmax, ymin, ymax)

class YoloReader(FileReader):
  classnames: List[str] = [
    '2c', '3c', '4c', '5c',
    '6c', '7c', '8c', '9c',
    '10c', 'Ac', 'Jc', 'Kc',
    'Qc',
    '2d', '3d', '4d', '5d',
    '6d', '7d', '8d', '9d',
    '10d', 'Ad', 'Jd', 'Kd',
    'Qd',
    '2h', '3h', '4h', '5h',
    '6h', '7h', '8h', '9h',
    '10h', 'Ah', 'Jh', 'Kh',
    'Qh',
    'As', '2s', '3s', '4s',
    '5s', '6s', '7s', '8s',
    '9s', '10s', 'Js', 'Ks',
    'Qs'
  ]

  @classmethod
  def read(cls, yolopath: str) -> Yolo:
    yolos = []
    for yolo_card in super().read(yolopath).decode('utf-8').splitlines():
      (index, *args) = map(float, yolo_card.split())
      yolos.append(Yolo(cls.classnames[int(index)], *args))
    return yolos

class YoloRecordProvider(RecordProvider):
  @classmethod
  def to_record(cls, image: Image.Image, yolos: List[Yolo]) -> tf.train.Example:
    return cls.record(cls._parse(image, yolos))

  @classmethod
  def _parse(cls, image: Image.Image, yolos: List[Yolo]):
    (xmins, xmaxs, ymins, ymaxs) = ([], [], [], [])
    (texts, labels) = ([], [])

    for (classname, xmin, xmax, ymin, ymax) in map(astuple, map(BBox.from_yolo, yolos)):
      texts.append(classname.encode('utf8'))
      labels.append(Classes[classname])
      xmins.append(xmin)
      xmaxs.append(xmax)
      ymins.append(ymin)
      ymaxs.append(ymax)

    return {
      'name': cls.u8('filename'.encode('utf8')),
      'encoded': cls.u8(ImageReader.encoded(image)),
      'format': cls.u8(image.format.encode('utf8')),
      'height': cls.i64(image.height),
      'width': cls.i64(image.width),
      'cards/bbox/xmin': cls.f32_list(xmins),
      'cards/bbox/xmax': cls.f32_list(xmaxs),
      'cards/bbox/ymin': cls.f32_list(ymins),
      'cards/bbox/ymax': cls.f32_list(ymaxs),
      'cards/class/text': cls.u8_list(texts),
      'cards/class/label': cls.i64_list(labels),
    }

#### Parsing Process

In [110]:
def yolo_pathfinder(extension: str) -> pathlib.Path:
  return pathlib.Path(Paths['yolo']).glob(f"*.{extension}")
def yolo_record(data: Tuple[Image.Image, Any]) -> tf.train.Example:
  return YoloRecordProvider.to_record(*data)
def find_yolo_record_count():
  return len(list(yolo_pathfinder('jpg')))

images = map(ImageReader.read_resized, yolo_pathfinder('jpg'))
yolos = map(YoloReader.read, yolo_pathfinder('txt'))

yolo_count = find_yolo_record_count()
yolo_records = map(yolo_record, zip(images, yolos))

## Saving parsed records into record TensorFlow file

In [ ]:
print("Started writing record...")
FeatureWriter.combine_write((xml_records, yolo_records), Paths['record'], total=xml_count + yolo_count)
print("Finished writing record...")

Started writing record...


#### Example

In [ ]:
def draw_bbox(image: Image.Image, record):
  feature = lambda name: record.features.feature[name].float_list.value

  image = ImageReader.decoded(record.features.feature['encoded'].bytes_list.value[0])
  open_cv_image = np.array(image)
  (width, height, _) = open_cv_image.shape
  for (classname, xmin, xmax, ymin, ymax) in zip(record.features.feature['cards/class/text'].bytes_list.value,
                                                 *map(feature, (
                                                     'cards/bbox/xmin', 'cards/bbox/xmax', 'cards/bbox/ymin',
                                                     'cards/bbox/ymax'))):
    (xmin, xmax, ymin, ymax) = map(int, (xmin * width, xmax * width, ymin * height, ymax * height))
    cv2.rectangle(open_cv_image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
  pl.plt.imshow(open_cv_image)
# print(f"TensorFlow version: {tf.__version__}")
# print(f"|> Training set row example <|")
# print(dfs['train'].head())
#
# print(f"|> Test set row example <|")
# print(dfs['test'].head())
# print(len(dfs['test']))
#
# print(f"|> Dataset classes <|")
# print(dfs['train']['class'].unique())
# print(f"len(dfs['train'])")
#
# print("|> Example paths <|")
# for cardpath in cardpaths.take(5): print(cardpath.numpy())

#### Features

## Augmenting.

### Read saved data from record

In [ ]:
test_data = FeatureReader.read(Paths['record'])
for data in test_data.take(2):
  example = tf.train.Example()
  example.ParseFromString(data.numpy())
  bytes_ = example.features.feature['encoded'].bytes_list.value[0]
  image = ImageReader.decoded(bytes_)
  image.show()

## Saving.

## Loading augmented.

# Model.

## Creating.

In [ ]:

model: keras.Sequential = keras.Sequential([
  tf.keras.layers.Rescaling(1. / 255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(len(Classes))
])

## Compiling

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy']
)

## Training.

In [ ]:
model.fit(
  dfs['train']
)

## Validating.

## Wniosking.